<b>Prepare Christmas cards with Gemini and Sheets</b>

Do you want a little help thinking of something personal to say on your Christmas cards? Obviously you shouldn't outsource everything, but maybe Gemini can help you think of what to say to the dozens of people you connect with.

In [37]:
#@title Authenticate and grant Colab access to your spreadsheets for this session

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
#@title Configure Gemini API key

#Access your Gemini API key

import google.generativeai as genai
from google.colab import userdata

gemini_api_secret_name = 'GOOGLE_API_KEY'  # @param {type: "string"}

try:
  GOOGLE_API_KEY=userdata.get(gemini_api_secret_name)
  genai.configure(api_key=GOOGLE_API_KEY)
except userdata.SecretNotFoundError as e:
   print(f'''Secret not found

This expects you to create a secret named {gemini_api_secret_name} in Colab

Visit https://makersuite.google.com/app/apikey to create an API key

Store that in the secrets section on the left side of the notebook (key icon)

Name the secret {gemini_api_secret_name}''')
   raise e
except userdata.NotebookAccessError as e:
  print(f'''You need to grant this notebook access to the {gemini_api_secret_name} secret in order for the notebook to access Gemini on your behalf.''')
  raise e
except Exception as e:
  # unknown error
  print(f"There was an unknown error. Ensure you have a secret {gemini_api_secret_name} stored in Colab and it's a valid key from https://makersuite.google.com/app/apikey")
  raise e

model = genai.GenerativeModel('gemini-pro')

#Create a spreadsheet

Create a new spreadsheet at [sheets.new](https://sheets.new) with the data Gemini needs: name, relationship, and (optionally) style of how you want to address that person (funny, tender, sincere, thankful, [blank inside](https://www.youtube.com/watch?v=auh_6pSwjHE), really any attribute you want.).

For this example we've made a small test spreadsheet for you that you can use.

In [45]:
#@title Enter the name of a spreadsheet to import

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1ZX5Q3BvhOegE33LaIA_zETouvJdQE7AgflkF6lyriCU/edit?usp=sharing" #@param {type:"string"}

worksheet = gc.open_by_url(spreadsheet_url).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows[0:5])

[['Name', 'Relationship', 'Style'], ['Jeremiah', 'Brother', 'Funny'], ['Mom', 'Mom', 'Sincere'], ['Brittney', 'Wife', 'Tender']]


In [35]:
#@title Use Gemini to suggest what to say on a Christmas note

rows[0].append('Note')

#skip the title row
for row in rows[1:]:
  prompt = "I'm writing Christmas cards. Can you suggest a short meaningful paragraph of text for a Christmas note? I'm writing a note to %s my %s and I'd like the style to be %s." % (row[0], row[1], row[2])

  response = model.generate_content(prompt)
  row.append(response.text)

#print out the results nicely
from IPython.display import Markdown
import textwrap

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

to_markdown(''.join([row[3]+'\n\n' for row in rows[1:]]))

> Jeremiah, you're like the 'R' in reindeer—without you, everything would be 'ein'deer! Let's sprinkle some gingerbread joy and have a holly jolly Christmas. Whatever you do, don't get 'elfed' up! Wishing you a season filled with warm hugs and mistletoes.
> 
> Merry Christmas, Mom! As we approach this special holiday, I want to pause and express my sincere gratitude for everything you've done for me. Your unwavering love, sacrificial spirit, and guidance have shaped me into the person I am today. Your presence in my life is a constant source of comfort, encouragement, and inspiration. May our home be filled with the warmth of love, laughter, and peace as we celebrate this joyous occasion. From the bottom of my heart, I wish you a Christmas filled with memories that will last a lifetime. Love, [Your Name]
> 
> To my loving wife Brittney, this festive season brings a glow in our hearts as we cherish the spirit of Christmas together. May the warmth of this holiday fill our home with love, laughter, and unforgettable moments. As we gather around the tree, let the twinkling lights and ornaments remind us of the joy and magic that this time of year brings. Merry Christmas, my dearest Brittney. I love you more than words can say.
> 


#(OPTIONAL) Write the notes back to your spreadsheet

This only works for a spreadsheet you own, not the example sheet!

In [18]:
#@title Write the notes back to your spreadsheet

#this chooses which column to write to
col = len(rows[0])
start_cell = chr(col + 64) + '1'

#write the column
try:
  worksheet.update(start_cell, [[row[3]] for row in rows])
except Exception as e:
  print('There was an error writing to your spreadsheet. Make sure you have permissions to edit the spreadsheet.')
  raise e

{'spreadsheetId': '1ZX5Q3BvhOegE33LaIA_zETouvJdQE7AgflkF6lyriCU',
 'updatedRange': 'Sheet1!D1:D4',
 'updatedRows': 4,
 'updatedColumns': 1,
 'updatedCells': 4}

#The notes are in!

Go check your spreadsheet in Drive - there's a new column with the notes Gemini generated. Happy Holidays!